In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
def load_data(test_size=0.2, random_state=42):

    # Load California Housing dataset
    data = fetch_california_housing()
    X, y = data.data, data.target

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    return X_train, X_test, y_train, y_test

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)

    return running_loss / len(loader.dataset)

In [ ]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            running_loss += loss.item() * X_batch.size(0)
    return running_loss / len(loader.dataset)

In [ ]:
if __name__ == '__main__':
    # Hyperparameters
    batch_size = 64
    lr = 1e-3
    epochs = 50
    test_size = 0.2

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load data
    X_train, X_test, y_train, y_test = load_data(test_size=test_size)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Model, loss, optimizer
    model = RegressionModel(input_dim=X_train.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss = evaluate(model, test_loader, criterion, device)
        print(f'Epoch {epoch:03d}: Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}')

    # Save the trained model
    torch.save(model.state_dict(), 'regression_model.pth')
    print('Training complete. Model saved to regression_model.pth')

Epoch 001: Train Loss: 1.4157 | Test Loss: 0.5344
Epoch 002: Train Loss: 0.4613 | Test Loss: 0.4712
Epoch 003: Train Loss: 0.4233 | Test Loss: 0.4225
Epoch 004: Train Loss: 0.4020 | Test Loss: 0.4150
Epoch 005: Train Loss: 0.4504 | Test Loss: 0.4004
Epoch 006: Train Loss: 0.3855 | Test Loss: 0.3787
Epoch 007: Train Loss: 0.3616 | Test Loss: 0.3687
Epoch 008: Train Loss: 0.3519 | Test Loss: 0.3621
Epoch 009: Train Loss: 0.3447 | Test Loss: 0.3499
Epoch 010: Train Loss: 0.3378 | Test Loss: 0.3456
Epoch 011: Train Loss: 0.3351 | Test Loss: 0.3377
Epoch 012: Train Loss: 0.3299 | Test Loss: 0.3355
Epoch 013: Train Loss: 0.3225 | Test Loss: 0.3260
Epoch 014: Train Loss: 0.3183 | Test Loss: 0.3237
Epoch 015: Train Loss: 0.3209 | Test Loss: 0.3180
Epoch 016: Train Loss: 0.3117 | Test Loss: 0.3148
Epoch 017: Train Loss: 0.3086 | Test Loss: 0.3264
Epoch 018: Train Loss: 0.3019 | Test Loss: 0.3213
Epoch 019: Train Loss: 0.3013 | Test Loss: 0.3109
Epoch 020: Train Loss: 0.3018 | Test Loss: 0.3075
